# `II. Field Simulation` 
This .pynb file reads `./inter_results/mesh_result.pkl` generated by `./I_Mesh_Processing.ipynb`, and simulates electric field distribution. The result is stored in `./inter_results/field_result.pkl`.

In this file we compute electric field or potential on some spatial points(called `grid`) for all electrode voltage configurations(i.e. one eletrode is 1V and the rest 0V).Major calculations calls `fastlap` C library which uses a pre-conditioned, adaptive, multipole-accelerated algorithm for solving Laplace problem. Two parameters control multipole acceleration.
+ num_mom, the number of multipole
+ num_lev, the number of levels in the hierarchical spatial decomposition.  
num_lev=1 means direct computation without multipole acceleration. See fastlap ug.pdf and README.rst.


In `run_job` function, `job` is `Configuration` instance and `grid` is discretirized spatial grid (not the mesh). The general workflow (also the routine of BEM method) are:  
1. `solve_singularities()` solves charge distributions by iterative methods to make it consistent with one electrode at 1V and others at 0V (unit potentials). `adapt_mesh()` refines meshes adaptively to achieve certain precision while solving sigulartities.
2. Compute potentials on given grid points by `simulate()`, based on the charge distributions gotten previously.
3. Potential data of each unit potential are saved seperately to a `Result` instance, and also export to VTK files.
4. Return total accumulated charge per electrode in the end.

notes:
* here we invoke `multiprocessing.set_start_method("fork")` for compatibility with python version 3.9
* This is the most time consuming part in our workflow. Running following codes in a `.py` file instead of `.ipynb` may be helpful. Closing other apps in your laptop is helpful. Using HPC is also an option. 
* for reference, this file in my laptop(macbook pro, 2 physical kernels) runs 800s
* This `.ipynb` file will generate a `./.vtk` file for intermediate files, which can be simply ignored.

## (1) define grid
First of all, we define some spatial points for observing the field, called grid



In [1]:
from time import time
import pickle
import numpy as np
from bem import Electrodes, Sphere, Mesh, Grid, Configuration, Result
import multiprocessing 
import sys
multiprocessing.set_start_method("fork")
sys.path.append('./helper_func')
from helper_functions import run_job, write_pickle

fout_name = './inter_results/mesh_result.pkl'
with open(fout_name,'rb') as f:
    mesh_unit,xl,yl,zl,mesh = pickle.load(f) # import results from mesh processing

# grid to evalute potential and fields atCreate a grid in unit of scaled length mesh_unit. Only choose the interested region (trap center) to save time.
n, s = 100, 0.002
Lx, Ly, Lz = 0.100,0.100,0.100 # in the unit of scaled length mesh_unit
sx, sy, sz = s, s, s

vtk_out = "./inter_results/.vtks/htrapf"
print("done")

# ni is grid point number, si is step size. Thus to fix size on i direction you need to fix ni*si.
nx, ny, nz = [2*np.ceil(L/2.0/s).astype('int') for L in (Lx, Ly, Lz)]
print("Size/l:", Lx, Ly, Lz)
print("Step/l:", sx, sy, sz)
print("Shape (grid point numbers):", nx, ny, nz)
grid = Grid(center=(xl,yl,zl), step=(sx, sy, sz), shape=(nx, ny, nz))
# Grid center (nx, ny ,nz)/2 is shifted to origin
print("Grid origin/l:", grid.get_origin())

/Users/hewenhao/opt/anaconda3/envs/ele39/lib/python3.9/site-packages/bem/triangulation.py:28: UserWarning: not tvtk found
  warnings.warn("not tvtk found")
/Users/hewenhao/opt/anaconda3/envs/ele39/lib/python3.9/site-packages/bem/electrostatics.py:30: UserWarning: not tvtk found
  warnings.warn("not tvtk found")


done
Size/l: 0.1 0.1 0.1
Step/l: 0.002 0.002 0.002
Shape (grid point numbers): 50 50 50
Grid origin/l: [ 0.1742   -0.052672  0.02732 ]


## (2) run jobs

evaluate electric potential for each configurations: one electrode at 1V, the rest 0V. Set `pmap` as `multiprocessing.Pool().map` for parallel computing. For serial map, use `map`.

In [3]:

jobs = list(Configuration.select(mesh,"DC.*","RF"))    # select() picks one electrode each time.


# run the different electrodes on the parallel pool
pmap = multiprocessing.Pool().map # parallel map
#pmap = map # serial map
t0 = time()

def run_map():
    list(pmap(run_job, ((jobs[i], grid, vtk_out) for i in range(len(jobs)))))
    print("Computing time: %f s"%(time()-t0))
    # run_job casts a word after finishing each electrode.

run_map()

fout = './inter_results/field_result'
write_pickle(vtk_out,fout,grid)

FLW-mulMatUp: no multipole acceleration at all
FLW-mulMatUp: no multipole acceleration at all
FLW-mulMatUp: no multipole acceleration at all
FLW-mulMatUp: no multipole acceleration at all
FLW-mulMatUp: no multipole acceleration at all
FLW-mulMatUp: no multipole acceleration at all
FLW-mulMatUp: no multipole acceleration at all
FLW-mulMatUp: no multipole acceleration at all
FLW-mulMatUp: no multipole acceleration at all
FLW-mulMatUp: no multipole acceleration at all
FLW-mulMatUp: no multipole acceleration at all
dump namedump name  ./inter_results/.vtks/htrapf_DC15.pkl./inter_results/.vtks/htrapf_DC13.pkl
finished job DC15

finished job DC13
FLW-mulMatUp: no multipole acceleration at all
dump name ./inter_results/.vtks/htrapf_DC11.pkl
finished job DC11
FLW-mulMatUp: no multipole acceleration at all
FLW-mulMatUp: no multipole acceleration at all
FLW-mulMatUp: no multipole acceleration at all
FLW-mulMatUp: no multipole acceleration at all
FLW-mulMatUp: no multipole acceleration at all
FLW